<a href="https://colab.research.google.com/github/racoope70/quant-trading-model-zoo/blob/main/PPO_QuantConnect_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "shimmy>=2.0.0"

In [ ]:
!pip -q install yfinance pywavelets transformers --upgrade

In [ ]:
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

In [ ]:
!apt-get update -qq && apt-get install -y \
    libcusolver11 libcusparse11 libcurand10 libcufft10 libnppig10 libnppc10 libnppial10 \
    cuda-toolkit-12-4

In [ ]:
!pip uninstall -y protobuf
!pip install protobuf==3.20.3


In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x \
    dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


In [ ]:
!pip install numba==0.60.0


In [ ]:
!pip install "stable-baselines3[extra]>=2.0.0" "gymnasium>=0.29" "shimmy>=2.0.0" \
  gym-anytrading yfinance pandas numpy scikit-learn xgboost joblib


In [ ]:
#!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib
#!pip install matplotlib scikit-learn pandas

In [ ]:
!pip install tensorflow==2.18.0

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory config failed: {e}")


In [ ]:
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [ ]:
#Step 7: authenticate with hugging face hub (optional)
#This allows for better access and avoids rate limits when downloading public models/datasets

# Authenticate with Hugging Face Hub
#notebook_login()

In [ ]:
from google.colab import files
uploaded = files.upload()   # run this cell and choose the file

In [ ]:

import os
if "Alpaca_keys.env.txt" in uploaded:
    os.rename("Alpaca_keys.env.txt", ".env")    # adjust if your uploaded name differs


In [ ]:
!pip -q install python-dotenv
from dotenv import load_dotenv
load_dotenv(".env")  # or your actual filename

import os
token = os.getenv("GITHUB_TOKEN")      # ← this reads from the .env
print("Token present?", bool(token))   # Do NOT print the token itself


In [ ]:
import sys, subprocess, os, json, time, warnings, logging
from datetime import datetime, timedelta, timezone
from typing import Dict, Any, List

#  Install dependencies (idempotent) 
def _pip(pkgs: List[str]):
    subprocess.check_call([sys.executable, "-m", "pip", "install", *pkgs])

try:
    import stable_baselines3, gymnasium, yfinance, requests, gym_anytrading, torch, numpy, pandas  # noqa: F401
except Exception:
    _pip([
        "torch==2.2.2",
        "stable-baselines3==2.2.1",
        "gymnasium==0.29.1",
        "gym-anytrading==1.3.4",
        "numpy==1.26.4",
        "pandas==2.2.2",
        "yfinance==0.2.40",
        "requests==2.32.3"
    ])

import numpy as np
import pandas as pd
import requests
import yfinance as yf
import gymnasium as gym
from gym_anytrading.envs import StocksEnv
from gymnasium.spaces import Box as GBox
import torch
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Logging 
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
L = logging.getLogger("producer")

warnings.filterwarnings("ignore")

#  Google Drive mount 
try:
    from google.colab import drive  # type: ignore
    if not os.path.ismount("/content/drive"):
        drive.mount("/content/drive")
except Exception:
    pass

#  Config 
RESULTS_ROOT   = "/content/drive/MyDrive/Results_May_2025"
MASTER_DIR     = os.path.join(RESULTS_ROOT, "ppo_models_master")  # where <prefix>_model.zip etc. live

# (Symbol -> prefix) from your saved models
PICKS: Dict[str, str] = {
    "UNH":  "ppo_UNH_window3",
    "TSLA": "ppo_TSLA_window2",
    "TMO":  "ppo_TMO_window3",
}

# yfinance fetch
YF_INTERVAL = "1m"
YF_DAYS     = 5

# Gist settings (token must be set by you in the session)
GITHUB_TOKEN  = os.environ.get("GITHUB_TOKEN", "").strip()
GIST_ID       = os.environ.get("GIST_ID", "").strip()   # leave empty on first run; we persist to file below
STATE_DIR     = "/content/drive/MyDrive/QuantConnect_Ready"
os.makedirs(STATE_DIR, exist_ok=True)
GIST_ID_PATH  = os.path.join(STATE_DIR, "live_signals_gist_id.txt")
GIST_FILENAME = "live_signals.json"
GIST_DESC     = "Live PPO signals for QC (Producer→Consumer)"

# Publish loop
RUN_LOOP  = False
SLEEP_SEC = 60

#  Feature function (fallback; replace with your real pipeline if desired) 
try:
    compute_enhanced_features  # type: ignore
except NameError:
    def compute_enhanced_features(df_in: pd.DataFrame) -> pd.DataFrame:
        # No-op fallback. Keep column names as standard OHLCV + whatever yfinance returns.
        # If your features.json expects extra engineered columns, create them here.
        return df_in

#  PPO Env (must match training time semantics) 
class ContinuousPositionEnv(StocksEnv):
    def __init__(self, df, frame_bound, window_size,
                 cost_rate=0.0002, slip_rate=0.0003,
                 k_alpha=0.20, k_mom=0.05, k_sent=0.0,
                 mom_source="denoised", mom_lookback=20,
                 min_trade_delta=0.01, cooldown=5, reward_clip=1.0):
        super().__init__(df=df.reset_index(drop=True), frame_bound=frame_bound, window_size=window_size)
        if isinstance(self.observation_space, gym.spaces.Box):
            self.observation_space = GBox(
                low=self.observation_space.low,
                high=self.observation_space.high,
                shape=self.observation_space.shape,
                dtype=self.observation_space.dtype,
            )
        self.action_space = GBox(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)
        self.cost_rate, self.slip_rate = float(cost_rate), float(slip_rate)
        self.k_alpha, self.k_mom = float(k_alpha), float(k_mom)
        self.k_sent = float(k_sent)
        self.mom_source, self.mom_lookback = str(mom_source), int(mom_lookback)
        self.min_trade_delta, self.cooldown = float(min_trade_delta), int(cooldown)
        self.reward_clip = float(reward_clip)
        self.nav, self.pos, self._last_trade_step = 1.0, 0.0, -self.cooldown

    def reset(self, **kwargs):
        out = super().reset(**kwargs)
        if isinstance(out, tuple): obs, info = out
        else: obs, info = out, {}
        self.nav, self.pos, self._last_trade_step = 1.0, 0.0, -self.cooldown
        info = info or {}
        info.update({"nav": self.nav, "pos": self.pos})
        return obs, info

    def _step_parent_hold(self):
        step_result = super().step(2)
        if len(step_result) == 5:
            obs, _env_rew, terminated, truncated, info = step_result
        else:
            obs, _env_rew, done, info = step_result
            terminated, truncated = bool(done), False
        return obs, terminated, truncated, info

    def _ret_t(self):
        cur  = float(self.df.loc[self._current_tick, 'Close'])
        prev = float(self.df.loc[max(self._current_tick - 1, 0), 'Close'])
        return 0.0 if prev <= 0 else (cur - prev) / prev

    def _mom_signal(self):
        if self.mom_source == "macd" and "MACD_Line" in self.df.columns:
            recent = self.df["MACD_Line"].iloc[max(self._current_tick-200,0):self._current_tick+1]
            return float(np.tanh(float(self.df.loc[self._current_tick, "MACD_Line"]) / (1e-6 + recent.std())))
        if "Denoised_Close" in self.df.columns and self._current_tick - self.mom_lookback >= 0:
            now  = float(self.df.loc[self._current_tick, "Denoised_Close"])
            then = float(self.df.loc[self._current_tick - self.mom_lookback, "Denoised_Close"])
            base = float(self.df.loc[max(self._current_tick - 1, 0), "Close"])
            slope = (now - then) / max(self.mom_lookback, 1)
            return float(np.tanh(10.0 * (slope / max(abs(base), 1e-6))))
        return 0.0

    def step(self, action):
        a = float(np.array(action).squeeze()); target_pos = float(np.clip(a, -1.0, 1.0))
        r_t = self._ret_t(); base_ret = self.pos * r_t
        changed = (abs(target_pos - self.pos) >= 0.01) and ((self._current_tick - self._last_trade_step) >= 5)
        delta_pos = (target_pos - self.pos) if changed else 0.0
        trade_cost = (0.0002 + 0.0003) * abs(delta_pos)
        rel_alpha  = base_ret - r_t
        mom_term   = self.pos * self._mom_signal()
        shaped     = base_ret + 0.20*rel_alpha + 0.05*mom_term - trade_cost
        reward     = float(np.clip(shaped, -1.0, 1.0))
        self.nav  *= (1.0 + base_ret - trade_cost)
        if changed:
            self.pos = target_pos; self._last_trade_step = self._current_tick
        obs, terminated, truncated, info = self._step_parent_hold()
        info = info or {}
        info.update({"ret_t": r_t, "nav": self.nav, "pos": self.pos,
                     "trade_cost": trade_cost, "base_ret": base_ret,
                     "rel_alpha": rel_alpha, "mom": self._mom_signal()})
        return obs, reward, terminated, truncated, info

# Helpers: artifacts, features, model load 
def _features_list_for(prefix: str) -> List[str]:
    fpath = os.path.join(MASTER_DIR, f"{prefix}_features.json")
    if os.path.exists(fpath):
        try:
            meta = json.load(open(fpath, "r"))
            feats = meta.get("features") or []
            if isinstance(feats, list):
                return [str(c) for c in feats]
        except Exception as e:
            L.warning(f"features.json read failed for {prefix}: {e}")
    return []

def _align_columns(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    feats = _features_list_for(prefix)
    if not feats:
        return df
    aligned = df.copy()
    for c in feats:
        if c not in aligned.columns:
            aligned[c] = 0.0
    # Optional: order features first, then anything extra
    ordered = [c for c in feats if c in aligned.columns] + [c for c in aligned.columns if c not in feats]
    return aligned[ordered]

def _check_artifacts(prefix: str) -> Dict[str, bool]:
    need = ["_model.zip", "_vecnorm.pkl", "_features.json", "_probability_config.json", "_model_info.json"]
    return {s: os.path.exists(os.path.join(MASTER_DIR, prefix + s)) for s in need}

def get_mu_sigma(model, obs):
    with torch.no_grad():
        obs_t, _     = model.policy.obs_to_tensor(obs)
        feats        = model.policy.extract_features(obs_t)
        latent_pi, _ = model.policy.mlp_extractor(feats)
        mean_actions = model.policy.action_net(latent_pi)
        log_std      = model.policy.log_std
        mu    = float(mean_actions.detach().cpu().numpy().squeeze())
        sigma = float(log_std.exp().detach().cpu().numpy().squeeze())
    return mu, sigma

def load_model_and_env(prefix: str):
    model_path = os.path.join(MASTER_DIR, f"{prefix}_model.zip")
    vec_path   = os.path.join(MASTER_DIR, f"{prefix}_vecnorm.pkl")
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Missing model: {model_path}")
    model = PPO.load(model_path, device="cpu")

    def make_env(df_window: pd.DataFrame):
        frame_bound = (50, len(df_window) - 3)
        e = DummyVecEnv([lambda: ContinuousPositionEnv(
            df=df_window, frame_bound=frame_bound, window_size=10,
            cost_rate=0.0002, slip_rate=0.0003,
            k_alpha=0.20, k_mom=0.05, k_sent=0.0,
            mom_source="denoised", mom_lookback=20,
            min_trade_delta=0.01, cooldown=5, reward_clip=1.0
        )])
        if os.path.exists(vec_path):
            e = VecNormalize.load(vec_path, e)
        e.training = False; e.norm_reward = False
        return e
    return model, make_env

# Data fetch + MultiIndex fix + predict 
def _flatten_yf_columns(df: pd.DataFrame, symbol: str) -> pd.DataFrame:
    """
    If yfinance returns a MultiIndex (e.g. ('Close','UNH')), flatten to plain columns:
      ('Close','UNH') -> 'Close'
      ('Adj Close','') -> 'Adj Close'
      otherwise -> f"{lvl0}_{lvl1}"
    """
    if isinstance(df.columns, pd.MultiIndex):
        new_cols = []
        for c0, c1 in df.columns.to_list():
            if c1 in ("", symbol):
                new_cols.append(c0)
            else:
                new_cols.append(f"{c0}_{c1}")
        df.columns = new_cols
    return df

def latest_df_for_symbol(symbol: str, horizon_days: int = YF_DAYS, interval: str = YF_INTERVAL) -> pd.DataFrame | None:
    end   = datetime.now(timezone.utc)
    start = end - timedelta(days=horizon_days)
    df = yf.download(symbol,
                     start=start.strftime("%Y-%m-%d"),
                     end=end.strftime("%Y-%m-%d"),
                     interval=interval,
                     progress=False,
                     auto_adjust=False)
    if df is None or df.empty:
        return None
    df = df.reset_index()
    df["Symbol"] = symbol
    df = _flatten_yf_columns(df, symbol)
    df = compute_enhanced_features(df)
    return df

def predict_latest(symbol: str, prefix: str) -> Dict[str, Any]:
    # artifacts present?
    missing = [k for k, v in _check_artifacts(prefix).items() if not v]
    if missing:
        return {"symbol": symbol, "prefix": prefix, "error": f"missing artifacts: {missing}"}

    model, make_env = load_model_and_env(prefix)

    live_df = latest_df_for_symbol(symbol)
    if live_df is None or len(live_df) < 100:
        return {"symbol": symbol, "prefix": prefix, "error": "no fresh data"}

    # align to training features if provided
    live_df = _align_columns(live_df, prefix)

    # window
    df_window = live_df.iloc[-2500:].reset_index(drop=True) if len(live_df) > 2500 else live_df.copy()

    # roll to last bar with HOLD
    env = make_env(df_window)
    obs = env.reset()
    if isinstance(obs, tuple):
        obs, _ = obs
    for _ in range(len(df_window) - 1):
        obs, _, dones, _ = env.step([np.array([0.0], dtype=np.float32)])
        if isinstance(dones, (np.ndarray, list)) and dones[0]:
            break

    action, _ = model.predict(obs, deterministic=True)
    mu, sigma = get_mu_sigma(model, obs)
    from math import erf, sqrt
    Phi = lambda x: 0.5 * (1.0 + erf(x / sqrt(2.0)))
    p_long = 1.0 - Phi((0.0 - mu) / max(sigma, 1e-6))

    a = float(np.array(action).squeeze())
    signal = "BUY" if a > 0.10 else ("SELL" if a < -0.30 else "HOLD")
    ts = df_window["Datetime"].iloc[-1] if "Datetime" in df_window.columns else None
    px = float(df_window["Close"].iloc[-1])

    return {
        "symbol": symbol,
        "prefix": prefix,
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "bar_ts": str(ts),
        "price": px,
        "action": a,
        "signal": signal,
        "confidence": abs(a),
        "p_long": float(p_long),
        "p_short": float(1.0 - p_long),
        "mu": float(mu),
        "sigma": float(sigma)
    }

#  Gist publisher 
def _headers(token: str) -> Dict[str, str]:
    return {"Authorization": f"token {token}"} if token else {}

def _load_saved_gist_id() -> str:
    if GIST_ID:
        return GIST_ID
    try:
        return open(GIST_ID_PATH, "r").read().strip()
    except Exception:
        return ""

def _save_gist_id(gid: str):
    try:
        with open(GIST_ID_PATH, "w") as f:
            f.write(gid)
    except Exception:
        pass

def publish_json_to_gist(payload: dict, filename: str, gist_id: str, token: str, desc: str) -> str:
    if not token:
        raise RuntimeError("GITHUB_TOKEN not set. In a cell above, run: os.environ['GITHUB_TOKEN'] = 'ghp_...'")

    files = {filename: {"content": json.dumps(payload, indent=2)}}
    if gist_id:
        r = requests.patch(f"https://api.github.com/gists/{gist_id}",
                           headers=_headers(token),
                           json={"files": files, "description": desc})
        if r.status_code // 100 != 2:
            raise RuntimeError(f"Gist update failed: {r.status_code} {r.text}")
        return gist_id
    else:
        r = requests.post("https://api.github.com/gists",
                          headers=_headers(token),
                          json={"files": files, "description": desc, "public": True})
        if r.status_code // 100 != 2:
            raise RuntimeError(f"Gist create failed: {r.status_code} {r.text}")
        new_id = r.json().get("id", "")
        if new_id:
            _save_gist_id(new_id)
        return new_id

def gist_raw_url(gist_id: str, filename: str) -> str:
    meta = requests.get(f"https://api.github.com/gists/{gist_id}").json()
    owner = (meta.get("owner") or {}).get("login", "anonymous")
    return f"https://gist.githubusercontent.com/{owner}/{gist_id}/raw/{filename}"

def run_once():
    results = []
    for sym, pref in PICKS.items():
        try:
            out = predict_latest(sym, pref)
            if out.get("error"):
                L.warning(f"{sym} -> {out['error']}")
            results.append(out)
        except Exception as e:
            L.exception(f"{sym} predict error: {e}")
            results.append({"symbol": sym, "prefix": pref, "error": str(e)})

    payload = {
        "generated_utc": datetime.now(timezone.utc).isoformat(),
        "valid_until_utc": (datetime.now(timezone.utc) + timedelta(minutes=3)).isoformat(),
        "producer": "colab-sb3",
        "interval": YF_INTERVAL,
        "models": results
    }

    gid = _load_saved_gist_id()
    gid = publish_json_to_gist(payload, filename=GIST_FILENAME, gist_id=gid, token=GITHUB_TOKEN, desc=GIST_DESC)
    raw = gist_raw_url(gid, GIST_FILENAME)
    print("Published:", f"https://gist.github.com/{gid}")
    print("RAW URL  :", raw)
    print("Preview  :", json.dumps(payload, indent=2)[:900], "...")

#  Sanity check + execute 
for sym, pref in PICKS.items():
    status = _check_artifacts(pref)
    missing = [k for k, v in status.items() if not v]
    print(f"{pref}: {'OK' if not missing else 'MISSING ' + str(missing)}")

# Run once (set RUN_LOOP=True above for periodic publishing)
run_once()
if RUN_LOOP:
    while True:
        try:
            run_once()
        except Exception as e:
            L.error(f"Publish error: {e}")
        time.sleep(SLEEP_SEC)


In [ ]:
import os, json, requests

RESULTS_ROOT     = "/content/drive/MyDrive/Results_May_2025/ppo_models_master/live_signals_gist_id.txt"
FINAL_MODEL_DIR  = os.path.join(RESULTS_ROOT, "ppo_models_master")
GIST_ID_PATH     = os.path.join(FINAL_MODEL_DIR, "live_signals_gist_id.txt")
FILENAME         = "live_signals.json"

with open(GIST_ID_PATH, "r") as f:
    gid = f.read().strip()

meta = requests.get(f"https://api.github.com/gists/{gid}").json()
owner = (meta.get("owner") or {}).get("login", "anonymous")
files = meta.get("files", {})
raw_url_api = files.get(FILENAME, {}).get("raw_url", "")

raw_url_stable = f"https://gist.githubusercontent.com/{owner}/{gid}/raw/{FILENAME}"

print("Gist page:", f"https://gist.github.com/{owner}/{gid}")
print("Raw (stable):", raw_url_stable)
print("Raw (API-provided, may include a revision hash):", raw_url_api)
